In [96]:
def get_df_from_json(data_url, record_path=['Data']):
    content = requests.get(data_url).json()
    df_all = pd.json_normalize(content,record_path=record_path)
    
    return df_all

In [107]:
# Scrape webpage
import io
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Start date and end date
start_date = '2022-10-14'
end_date = '2022-10-15'



In [98]:
# Get river data
river_data_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=35946&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
flow_df_all = get_df_from_json(river_data_url)
flow_df = flow_df_all[["TimeStamp", "Value"]]
flow_df = flow_df.rename(columns={'Value': 'FlowRate'})

print(flow_df.head())

              TimeStamp   FlowRate
0  2022-10-15T00:00:00Z  40.598459
1  2022-10-14T23:55:00Z  40.598459
2  2022-10-14T23:50:00Z  40.429302
3  2022-10-14T23:45:00Z  40.361712
4  2022-10-14T23:40:00Z  40.159192


In [99]:
# Get Lake levels
lake_level_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=32419&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
lake_level_df_all = get_df_from_json(lake_level_url)
lake_level_df = lake_level_df_all[["TimeStamp", "Value"]]
lake_level_df = lake_level_df.rename(columns={'Value': 'LakeLevel'})
print(lake_level_df.head())


              TimeStamp  LakeLevel
0  2022-10-15T00:00:00Z    279.221
1  2022-10-14T23:55:00Z    279.220
2  2022-10-14T23:50:00Z    279.221
3  2022-10-14T23:45:00Z    279.221
4  2022-10-14T23:40:00Z    279.223


In [100]:
# Get rainfall at Lake Rotoiti

rainfall_url = 'https://envdata.boprc.govt.nz/Data/DatasetGrid?dataset=47928&sort=TimeStamp-desc&page=1&group=&filter=&interval=Custom&timezone=720&date={}&endDate={}&calendar=1&alldata=false'.format(start_date, end_date)
rainfall_url_all = get_df_from_json(rainfall_url)
rainfall_url = rainfall_url_all[["TimeStamp", "Value"]]
rainfall_url = rainfall_url.rename(columns={'Value': 'Rainfall'})
rainfall_url

,TimeStamp,Rainfall
0,2022-10-15T00:00:00Z,0.0
1,2022-10-14T23:00:00Z,0.0
2,2022-10-14T22:00:00Z,0.0
3,2022-10-14T21:00:00Z,0.0
4,2022-10-14T20:00:00Z,0.0
5,2022-10-14T19:00:00Z,0.0
6,2022-10-14T18:00:00Z,0.0
7,2022-10-14T17:00:00Z,0.0
8,2022-10-14T16:00:00Z,0.0
9,2022-10-14T15:00:00Z,0.0
